In [1]:
import pandas as pd
import ast
from tqdm import tqdm

In [2]:
ospath = '/Users/nguyenduykhang/Documents/Projects/HPCC-BusAlgos/'
path_route_df = ospath + 'data/all_routes.csv'

all_routes = pd.read_csv(path_route_df)
num_routes = len(all_routes)

In [4]:
hub_data_path = "/Users/nguyenduykhang/Documents/Projects/HPCC-BusAlgos/data/bus-lines"
bus_routes_dict = dict()
for route in tqdm(range(num_routes)):
    routeNo = all_routes.iloc[route]['RouteNo']
    for dir in [0, 1]:
        routeName = f"{routeNo}_{dir}"
        bus_hub = pd.read_csv(f"{hub_data_path}/{routeNo}/{dir}_hub_nodes.csv")
        hub_list = list(bus_hub.StopId)
        bus_routes_dict[routeName] = hub_list

100%|██████████| 128/128 [02:21<00:00,  1.11s/it]


**Version 1: BFS search**

In [6]:
bus_routes_dict.items()

dict_items([('1_0', [33, 79, 76, 2028, 120, 121, 122, 126, 14, 137, 138, 8]), ('1_1', [8, 2, 3, 5, 11, 14, 18, 25, 26, 27, 7266, 1256, 31, 33]), ('3_0', [35, 7277, 7266, 1256, 32, 31, 34, 43, 47, 50, 56, 60, 72, 75]), ('3_1', [75, 3255, 208, 212, 219, 224, 225, 232, 234, 233, 237, 79, 76, 2028, 7269, 7273, 7274, 35]), ('4_0', [35, 7277, 7266, 1256, 4273, 4274, 244, 246, 252, 256, 258, 268, 267, 271, 273, 1115, 1239, 166]), ('4_1', [166, 169, 174, 175, 177, 187, 186, 192, 195, 200, 199, 7273, 7274, 35]), ('5_0', [8, 897, 1050, 281, 285, 288, 3164, 287, 291, 294, 293, 297, 299, 301, 304, 307, 314, 318, 315, 320, 7208]), ('5_1', [7208, 339, 341, 342, 347, 349, 355, 358, 360, 361, 366, 367, 374, 371, 378, 377, 380, 89, 90, 384, 3180, 2978, 393, 1017, 8]), ('6_0', [8, 437, 439, 440, 466, 467, 287, 291, 294, 293, 297, 476, 477, 482, 481, 483, 486, 493, 490, 497, 494, 7159, 495, 394]), ('6_1', [394, 396, 398, 401, 402, 410, 412, 415, 417, 418, 420, 421, 367, 374, 371, 378, 377, 380, 89, 90, 4

In [7]:
from collections import defaultdict

bus_stops = defaultdict(set)
adj_list = defaultdict(set)

for route, stops in bus_routes_dict.items():
    for stop in stops:
        bus_stops[stop].add(route)
        
for stops in bus_stops.values():
    for i in stops:
        for j in stops:
            if i != j:
                adj_list[i].add(j)

In [ ]:
from collections import defaultdict, deque

class Bus_Route_Search: #BFS search
    def __init__(self, all_bus_routes):
        self.bus_routes = all_bus_routes
        self.bus_stops = defaultdict(set)
        self.adj_list = defaultdict(set)
        
    def bus_graph(self):
        for route, stops in self.bus_routes.items():
            for stop in stops:
                self.bus_stops[stop].add(route)
                
        for stops in self.bus_stops.values():
            for i in stops:
                for j in stops:
                    if i != j:
                        self.adj_list[i].add(j)
        
    def bus_path_findding(self, start_stop, target_stop):
        if start_stop == target_stop:
            return 0, []
          
        start_routes = self.bus_stops[start_stop]
        target_routes = self.bus_stops[target_stop]  
        print(f"start_stop: {start_stop}")
        print(f"target_stop: {target_stop}")
        
        if not start_routes or not target_routes:
            return -1, []
        
        queue = deque([(route, 1, [route]) for route in start_routes])
        visited = set()
        
        while queue:
            current_route, dis, route = queue.popleft()
    
            if current_route in target_routes:
                # print(f"-------->>  ", dis, route)
                return dis, route
            
            visited.add(current_route)
            for neighbor_route in self.adj_list[current_route]:
                if neighbor_route not in visited:
                    queue.append((neighbor_route, dis+1, route + [neighbor_route]))
        
        return -2, []
    
    def switch_stops(self, bus_path):
        switch_stops = set()
        for route in range(len(bus_path) - 1):
            current_route = bus_path[route]
            next_route= bus_path[route + 1]
            switch_stops.update(set(self.bus_routes[next_route]) & set(self.bus_routes[current_route]))
            
        return list(switch_stops)

In [ ]:
import pandas as pd
import ast
from tqdm import tqdm

file_path = '/Users/nguyenduykhang/Documents/Projects/HCMC-Bus-Network/data/all_bus_routes.csv'

def route_adj_list():
    df = pd.read_csv(file_path)
    df['routeNoTy'] = df['routeNo'].astype(str) + '-' + df['routeType'].astype(str)
    num_routes = len(df) -1
    
    bus_routes = dict()
    for route in tqdm(range(num_routes)):
        routeNoTy = df.iloc[route]['routeNoTy']
        bus_stops = df.iloc[route]['bus_stops']
        bus_routes[routeNoTy] = ast.literal_eval(bus_stops)
        
    return bus_routes